## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,65.26,88,75,1.99,broken clouds
1,1,Langsa,ID,4.4683,97.9683,83.28,75,99,4.63,overcast clouds
2,2,Kapaa,US,22.0752,-159.3190,78.78,78,40,1.99,scattered clouds
3,3,Saint-Philippe,RE,-21.3585,55.7679,81.99,82,95,15.52,overcast clouds
4,4,Dikson,RU,73.5069,80.5464,-12.95,96,100,16.55,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 70
What is the maximun temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Langsa,ID,4.4683,97.9683,83.28,75,99,4.63,overcast clouds
2,2,Kapaa,US,22.0752,-159.3190,78.78,78,40,1.99,scattered clouds
3,3,Saint-Philippe,RE,-21.3585,55.7679,81.99,82,95,15.52,overcast clouds
6,6,Santa Rita,BR,-7.1139,-34.9781,75.87,88,20,4.61,few clouds
9,9,Geraldton,AU,-28.7667,114.6000,78.13,31,40,20.71,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
cities_preferred_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

621


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Langsa,ID,4.4683,97.9683,83.28,75,99,4.63,overcast clouds
2,2,Kapaa,US,22.0752,-159.3190,78.78,78,40,1.99,scattered clouds
3,3,Saint-Philippe,RE,-21.3585,55.7679,81.99,82,95,15.52,overcast clouds
6,6,Santa Rita,BR,-7.1139,-34.9781,75.87,88,20,4.61,few clouds
9,9,Geraldton,AU,-28.7667,114.6000,78.13,31,40,20.71,scattered clouds


In [9]:
clean_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Langsa,ID,83.28,overcast clouds,4.4683,97.9683,
2,Kapaa,US,78.78,scattered clouds,22.0752,-159.3190,
3,Saint-Philippe,RE,81.99,overcast clouds,-21.3585,55.7679,
6,Santa Rita,BR,75.87,few clouds,-7.1139,-34.9781,
9,Geraldton,AU,78.13,scattered clouds,-28.7667,114.6000,
10,Guarapari,BR,78.69,clear sky,-20.6667,-40.4975,
13,Airai,TL,71.83,broken clouds,-8.9266,125.4092,
18,Hobart,AU,81.01,few clouds,-42.8794,147.3294,
19,Geraldton,AU,78.13,scattered clouds,-28.7667,114.6000,
21,Saint-Philippe,RE,81.99,overcast clouds,-21.3585,55.7679,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
       
    except (IndexError):
        print("Cannot find hotel...skipping.")  

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Langsa,ID,83.28,overcast clouds,4.4683,97.9683,OYO 2964 Hotel Ridho
2,Kapaa,US,78.78,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Saint-Philippe,RE,81.99,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
6,Santa Rita,BR,75.87,few clouds,-7.1139,-34.9781,Infinity Motel
9,Geraldton,AU,78.13,scattered clouds,-28.7667,114.6000,Broadwater Mariner Resort
10,Guarapari,BR,78.69,clear sky,-20.6667,-40.4975,Hotel Porto do Sol
18,Hobart,AU,81.01,few clouds,-42.8794,147.3294,Mantra on Collins Hobart
19,Geraldton,AU,78.13,scattered clouds,-28.7667,114.6000,Broadwater Mariner Resort
21,Saint-Philippe,RE,81.99,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
22,New Norfolk,AU,80.65,clear sky,-42.7826,147.0587,The Shingles Riverside Cottages


In [13]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure

figure = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
figure.add_layer(marker_layer)
figure

Figure(layout=FigureLayout(height='420px'))